# Clean Data

In [1]:
import pandas as pd
import numpy as np
import datetime
import copy
import os
import re

today = datetime.date.today()

In [2]:
chunks = pd.read_csv('../raw_data/desktop_scraped_data.csv', chunksize=10000)
# chunks = pd.read_csv('../desktop_raw_data/desktop_scraped_data.csv')  # data copied from desktop



df = pd.concat(chunks, ignore_index=True)

df.to_csv('../backup_data/backup_data_{}.csv'.format(len(os.listdir('../backup_data/'))))  # create back up of csv

df.shape

(4409663, 15)

In [5]:
try:
    df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    pass

In [6]:
df.head()

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title
0,jonashendrickx,NaN,347,/r/thinkpad/comments/74drfl/thinkpad_25_discus...,0,NaN,/r/thinkpad/comments/74drfl/thinkpad_25_discus...,t3_74drfl,1507178424000,87.0,1.509491e+09,r/thinkpad,3637.0,21512.0,ThinkPad 25 Discussion Thread (Release Oct 5) ...
1,MercGaberkles,NaN,4,https://i.redd.it/bwujg5tvn7vz.jpg,0,NaN,/r/thinkpad/comments/79xuvy/happy_halloween_rt...,t3_79xuvy,1509475415000,50.0,1.509491e+09,r/thinkpad,3637.0,21512.0,Happy Halloween r/ThinkPad! (i.redd.it)
2,dan988,NaN,68,https://i.redd.it/vqqssqp9k5vz.jpg,0,NaN,/r/thinkpad/comments/79v62u/swapped_out_an_xps...,t3_79v62u,1509450227000,63.0,1.509491e+09,r/thinkpad,3637.0,21512.0,Swapped out an XPS13 for this! (i.redd.it)
3,MUnderwoodBarcode,NaN,5,https://imgur.com/gallery/pxGGU,0,NaN,/r/thinkpad/comments/79yy81/spotted_joe_rogans...,t3_79yy81,1509484723000,NaN,1.509491e+09,r/thinkpad,3637.0,21512.0,Spotted Joe Rogan's TP (imgur.com)
4,Hotshot9146,NaN,3,https://imgur.com/gallery/cBj6E,0,NaN,/r/thinkpad/comments/79z1d7/mint_x61_for_now_ʖ...,t3_79z1d7,1509485487000,NaN,1.509491e+09,r/thinkpad,3637.0,21512.0,"Mint X61... for now ( ͡~ ͜ʖ ͡°) $60 for it, de..."


In [7]:
print('Subreddits: {}'.format(len(df['subreddit'].unique())))
print(df['subreddit'].unique())

Subreddits: 2923
['r/thinkpad' 'r/IWantOut' 'r/simpleliving' ..., 'r/Eve' 'r/ravens'
 'r/Emma_Roberts']


In [8]:
df = df.drop_duplicates(['author', 'title'], keep='last')
df = df[df['author'] != 'AutoModerator']  # remove posts from the moderator bot
df.shape

(2562674, 15)

In [9]:
df.describe()

,comment_karma,comments,crossposts,karma,post_time,score,scrape_time,subreddit_rank,subscriptions
count,0.0,2.562674e+06,2.562674e+06,0.0,2.562674e+06,2.555398e+06,2.562674e+06,2.562229e+06,2.562229e+06
mean,NaN,1.639949e+01,1.746613e-03,NaN,1.501823e+12,1.444772e+02,1.509584e+09,4.831177e+03,1.928882e+05
std,NaN,8.898136e+01,4.929062e-02,NaN,1.306763e+10,1.208978e+03,4.668304e+04,5.301881e+03,1.336683e+06
min,NaN,0.000000e+00,0.000000e+00,NaN,1.274456e+12,0.000000e+00,1.509491e+09,2.000000e+00,3.400000e+01
25%,NaN,2.000000e+00,0.000000e+00,NaN,1.501017e+12,4.000000e+00,1.509541e+09,1.428000e+03,1.070100e+04
50%,NaN,6.000000e+00,0.000000e+00,NaN,1.506293e+12,1.400000e+01,1.509597e+09,3.312000e+03,2.418700e+04
75%,NaN,1.400000e+01,0.000000e+00,NaN,1.508476e+12,5.200000e+01,1.509624e+09,6.401000e+03,6.351200e+04
max,NaN,3.488200e+04,9.000000e+00,NaN,1.509647e+12,1.621870e+05,1.509648e+09,1.676510e+05,1.826758e+07


In [10]:
df['author'].value_counts().head(10)

Lol33ta                2364
RyanSmith              2130
Reddit__PI             1323
KevlarYarmulke         1197
jaykirsch              1181
love_the_heat           985
GiantWithDwarfism       952
NegativeWithGoldBot     940
MyRSSbot                923
ChannelBot              921
Name: author, dtype: int64

In [11]:
df['title'] = [re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", title)).replace('()', '') for title in df['title'].values]

In [13]:
from emoji import UNICODE_EMOJI

type(UNICODE_EMOJI)

def has_emoji(text):
    emoji = 0
    try:
        for i in text:
            if i in UNICODE_EMOJI.keys():
                emoji = 1
                break
    except:
        pass
    return emoji
        
    
df['has_emoji'] = df['title'].apply(has_emoji)

In [14]:
df[df['has_emoji'] == 1].head(1)

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title,has_emoji
4689,antibes18,NaN,0,https://i.redd.it/5sfgo551s6vz.jpg,0,NaN,/r/labrador/comments/79wlkr/happy_halloween/,t3_79wlkr,1509464717000,3.0,1.509491e+09,r/labrador,4542.0,16682.0,"""Happy Halloween""🐾🐾🐾",1


In [15]:
def outside_content(content):
    outside = 0
    if content[1] != 'r':
        outside = 1
    return outside

df['outside_content'] = df['content'].apply(outside_content)

In [16]:
df.head()

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title,has_emoji,outside_content
996,Jason_w233,NaN,10,/r/thinkpad/comments/74tow9/x270/,0,NaN,/r/thinkpad/comments/74tow9/x270/,t3_74tow9,1507363867000,0.0,1.509491e+09,r/thinkpad,3637.0,21512.0,X270,0,0
997,Dav2481,NaN,3,/r/thinkpad/comments/74mwr5/what_are_the_diffe...,0,NaN,/r/thinkpad/comments/74mwr5/what_are_the_diffe...,t3_74mwr5,1507288004000,8.0,1.509491e+09,r/thinkpad,3637.0,21512.0,What are the differences between Think Pad mod...,0,0
998,Rjania2,NaN,7,/r/thinkpad/comments/74prd5/seroius_salt_dont_...,0,NaN,/r/thinkpad/comments/74prd5/seroius_salt_dont_...,t3_74prd5,1507316431000,1.0,1.509491e+09,r/thinkpad,3637.0,21512.0,"Seroius salt, don't read without adequate water!",0,0
1006,themeanderingalley,NaN,0,https://www.youtube.com/watch?v=3-Q6KfHj-KM&t=1s,0,NaN,/r/IWantOut/comments/79z7gs/a_picturesque_banf...,t3_79z7gs,1509487018000,NaN,1.509491e+09,r/IWantOut,1061.0,85508.0,A picturesque Banff! Fall in love with the Can...,0,1
1016,Put_it_in_some_rice,NaN,20,/r/IWantOut/comments/79ofj0/brexit_want_to_lea...,0,NaN,/r/IWantOut/comments/79ofj0/brexit_want_to_lea...,t3_79ofj0,1509377240000,13.0,1.509491e+09,r/IWantOut,1061.0,85508.0,Want to leave the UK to France and need advice.,0,0


In [17]:
def is_question(title):
    question = 0
    if '?' in title:
        question = 1
    return question

df['is_question'] = df['title'].apply(is_question)

In [18]:
df.head()

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,scrape_time,subreddit,subreddit_rank,subscriptions,title,has_emoji,outside_content,is_question
996,Jason_w233,NaN,10,/r/thinkpad/comments/74tow9/x270/,0,NaN,/r/thinkpad/comments/74tow9/x270/,t3_74tow9,1507363867000,0.0,1.509491e+09,r/thinkpad,3637.0,21512.0,X270,0,0,0
997,Dav2481,NaN,3,/r/thinkpad/comments/74mwr5/what_are_the_diffe...,0,NaN,/r/thinkpad/comments/74mwr5/what_are_the_diffe...,t3_74mwr5,1507288004000,8.0,1.509491e+09,r/thinkpad,3637.0,21512.0,What are the differences between Think Pad mod...,0,0,1
998,Rjania2,NaN,7,/r/thinkpad/comments/74prd5/seroius_salt_dont_...,0,NaN,/r/thinkpad/comments/74prd5/seroius_salt_dont_...,t3_74prd5,1507316431000,1.0,1.509491e+09,r/thinkpad,3637.0,21512.0,"Seroius salt, don't read without adequate water!",0,0,0
1006,themeanderingalley,NaN,0,https://www.youtube.com/watch?v=3-Q6KfHj-KM&t=1s,0,NaN,/r/IWantOut/comments/79z7gs/a_picturesque_banf...,t3_79z7gs,1509487018000,NaN,1.509491e+09,r/IWantOut,1061.0,85508.0,A picturesque Banff! Fall in love with the Can...,0,1,0
1016,Put_it_in_some_rice,NaN,20,/r/IWantOut/comments/79ofj0/brexit_want_to_lea...,0,NaN,/r/IWantOut/comments/79ofj0/brexit_want_to_lea...,t3_79ofj0,1509377240000,13.0,1.509491e+09,r/IWantOut,1061.0,85508.0,Want to leave the UK to France and need advice.,0,0,0


In [19]:
df['scrape_time'] = df['scrape_time'].apply(datetime.datetime.fromtimestamp)

In [20]:
df['post_time'] = df['post_time']/1000
df['post_time'] = df['post_time'].apply(datetime.datetime.fromtimestamp)

df['elapsed_time'] = df['scrape_time'] - df['post_time']
df['minutes_since_post'] = df['elapsed_time'].dt.total_seconds()/60

In [21]:
defaults = ['r/announcements/', 'r/Art', 'r/AskReddit', 'r/askscience', 'r/aww', 'r/blog', 'r/books', 'r/creepy',
            'r/dataisbeautiful', 'r/DIY', 'r/Documentaries', 'r/EarthPorn', 'r/explainlikeimfive', 'r/food', 
            'r/funny', 'r/Futurology', 'r/gadgets', 'r/gaming', 'r/GetMotivated', 'r/gifs', 'r/history', 'r/IAmA',
            'r/InternetIsBeautiful', 'r/Jokes', 'r/LifeProTips', 'r/listentothis', 'r/mildlyinteresting', 'r/movies',
            'r/Music', 'r/news', 'r/nosleep', 'r/nottheonion', 'r/OldSchoolCool', 'r/personalfinance', 'r/philosophy',
            'r/photoshopbattles', 'r/pics', 'r/science', 'r/Showerthoughts', 'r/space', 'r/sports', 'r/television',
            'r/tifu', 'r/todayilearned', 'r/UpliftingNews', 'r/videos', 'r/worldnews']

def default_subreddit(subreddit):
    is_default = 0
    if subreddit in defaults:
        is_default = 1
    return is_default

df['default_subreddit'] = df['subreddit'].apply(default_subreddit)

In [22]:
df[df['default_subreddit'] == 1].head(1)

,author,comment_karma,comments,content,crossposts,karma,link,post_id,post_time,score,...,subreddit,subreddit_rank,subscriptions,title,has_emoji,outside_content,is_question,elapsed_time,minutes_since_post,default_subreddit
42448,AnotherSolo,NaN,259,http://www.slashfilm.com/what-we-do-in-the-sha...,0,NaN,/r/television/comments/79wgcn/what_we_do_in_th...,t3_79wgcn,2017-10-31 11:24:11,8776.0,...,r/television,21.0,13818267.0,/r/all‘What We Do in the Shadows’ TV Show in D...,0,1,0,08:01:43.371092,481.722852,1


In [23]:
if not os.path.isfile('../cleaned_data/cleaned_data.csv'):
    print("\nNEW FILE\n")
    df.to_csv('../cleaned_data/cleaned_data.csv', index=False)
else:
    df.to_csv('../cleaned_data/cleaned_data.csv', mode='a', index=False, header=False)


NEW FILE



In [24]:
clean_data = pd.read_csv('../cleaned_data/cleaned_data.csv')

clean_data.sort_values('scrape_time', inplace=True)

clean_data.drop_duplicates('post_id', keep='last', inplace=True)

In [25]:
clean_data.shape

(2561982, 21)

In [26]:
clean_data.to_csv('/data/reddit_data.csv', index=False)